## Optimization 

이전 단계에서 파라미터의 기울기 값을 추적하는 방법을 알았다. 
이번 단계에서는 추적한 기울기값을 적용하는 방법을 알아본다. 


In [31]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

데이터셋을 load하고 신경망을 구성한다. 

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

하이퍼 파라미터 설정 

In [28]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습시 기울기계산과 최적화단계를 설정한다. 
tensorflow와 비교해서 다른점은 tensorflow는 fit메소드를 통해 손쉽게 학습이 가능했지만 pytorch의 경우 이러한 방법으로 하나하나 코드를 작성해 주어야 한다는 점이다. 또한 매 epoch 마다 출력되는 loss값이나 acc값에 대한 출력코드도 따로 작성해주어 나타나게 해야 한다. 

In [32]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300625  [    0/60000]
loss: 2.303093  [ 6400/60000]
loss: 2.305039  [12800/60000]
loss: 2.301892  [19200/60000]
loss: 2.302276  [25600/60000]
loss: 2.302582  [32000/60000]
loss: 2.293050  [38400/60000]
loss: 2.299089  [44800/60000]
loss: 2.299665  [51200/60000]
loss: 2.292407  [57600/60000]
Test Error: 
 Accuracy: 10.9%, Avg loss: 2.298772 

Done!
